# Realizo la clasificacion usando Random Forest

## Tabla de resultados

In [1]:
#PCA(n)    test_size(n)         Jupyter            Kaggle
#Sin PCA       0.1               ~0.86
#   1          0.1               ~0.75            0.84521
#   2          0.1               ~0.87
#   3          0.1               ~0.88
#   4          0.1               ~0.87
#   5          0.1               ~0.87

## Observaciones

In [2]:
#El resultado no es fijo sino que va variando.
#EJEMPLO:
#                      PCA(n)   test_size(n)         Jupyter          
#Primer  intento        Sin PCA      0.1         0.8611554774568408 |
#Segundo intento        Sin PCA      0.1         0.8606502763962325 |=> ~0.86
#Tercer  intento        Sin PCA      0.1         0.8601011448086145 |

#Notar que si bien hay variabilidad, esta es muy pequenia.

## Importo librerias necesarias

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#Random Forest
from sklearn.ensemble import RandomForestClassifier
#PCA
from sklearn.decomposition import PCA

## Leo todos los datos

In [4]:
df = pd.read_csv('../data/final_data/entrenamiento_postulaciones_vistas4.csv', encoding='utf-8')
df.head()

,idaviso,idpostulante,timestamp,valor_educacion,sexo,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo
0,1111556097,365446,7.038007e+06,17,1,0,0,0,0,0,0,0,0,1
1,1112370711,365446,7.476885e+06,17,1,6996,0,0,0,0,0,45,14,1
2,1112326655,365446,9.809277e+06,17,1,11827,0,0,0,0,0,4,33,1
3,1112350182,365446,8.856385e+06,17,1,2141,0,0,0,0,0,31,38,1
4,1112383837,365446,7.563903e+06,17,1,6942,0,0,0,0,0,75,46,1


In [5]:
n_features = 13

In [6]:
X = np.array(df.iloc[:, 0:n_features])
y = np.array(df['se_postulo']) 

## Aplico PCA

In [7]:
pca = PCA(n_components=1)
pca.fit(X)
X = pca.transform(X)

## Realizo el Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

## Defino el clasificador

In [9]:
clf = RandomForestClassifier()

## Entreno el clasificador

In [10]:
clf.fit(X_train, y_train);

## Reviso el score

In [11]:
clf.score(X_test, y_test)

0.7499359346481113

## Predigo con todos los datos

In [12]:
X_full_data = X
pred_full_data = clf.predict(X_full_data)
df['sepostulo'] = pred_full_data

## Armo .csv para subir a Kaggle

In [13]:
top100 = pd.read_csv("../test_final_100k.csv", encoding='utf-8')
id_postulantes = pd.read_csv('../data/final_data/entrenamiento_id_postulantes.csv', encoding='utf-8')
top100_full = pd.merge(top100, id_postulantes, how='left', on='idpostulante')
top100_full.drop('idpostulante', inplace=True, axis=1)
top100_full.rename({'idpostulante2':'idpostulante'}, inplace=True, axis=1)
top100_full = pd.merge(top100_full, df, how='left', on=['idaviso','idpostulante'])
top100_full = top100_full[['id', 'se_postulo', 'sepostulo']]
top100_full.fillna(0, inplace=True)
top100_full['se_postulo'] = top100_full['se_postulo'].astype('int')
top100_full['sepostulo'] = top100_full['sepostulo'].astype('int')
top100_full.loc[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1), 'sepostulo'] = 1
top100_full[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1)]['sepostulo']
top100_full.drop('se_postulo', axis=1, inplace=True)
top100_full.to_csv('../data/final_data/top100k.csv', encoding='utf-8', index=False)